# Geocoding HDB Property Info Dataset using OneMap API

In [ ]:
# !pip install requests time pandas tqdm geopandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 7.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 21.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 49.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.9 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.3 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
#Import necessary libraries
import requests
import time
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

/Users/wanlin/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generating HDB Coordinates

In [ ]:
#Define functions for API calling and geocoding
def call_loop(url_given, attempts=3, delay=5):
    """
    Call the given URL with retries if response status is not 200.
    """
    response_status_code = 1
    for _ in range(attempts + 1):
        if response_status_code != 200:
            if response_status_code != 1:
                time.sleep(delay)
            response = requests.get(url_given)
            response_status_code = response.status_code
        else:
            return response
    print(f"For URL: {url_given}")
    print(f"Response Status Code: {response_status_code}")
    return None

def collect_results(full_add, attempts=3, delay=5):
    """
    Collects all the result from OneMap API into a list of dictionaries.
    Uses the full address (with spaces replaced by '+') to form the URL.
    """
    url_p1 = "https://www.onemap.gov.sg/api/common/elastic/search?searchVal=+"
    url_p2 = "+&returnGeom=Y&getAddrDetails=Y"
    final_url = url_p1 + full_add + url_p2
    
    results = []
    response = call_loop(final_url, attempts, delay)
    if response is not None:
        # Using response.json() is safer than eval(response.text)
        feedback = response.json()
        found = feedback.get('found', 0)
        totalpages = feedback.get('totalNumPages', 1)
    else:
        print(f"Exited collect_results function as there is no response for address: {full_add}")
        return results  # Returns empty list
    
    if found == 0:
        print(f"No results found for address: {full_add}")
        return results  # No results found
    else:
        results = feedback.get('results', [])
        # If there are multiple pages, get additional results
        if found > 1 and totalpages > 1:
            for x in range(totalpages - 1):
                page_num = x + 2
                url_with_page = final_url + f"&pageNum={page_num}"
                temp_response = call_loop(url_with_page, attempts, delay)
                if temp_response is not None:
                    temp_feedback = temp_response.json()
                    temp_results = temp_feedback.get('results', [])
                    results.extend(temp_results)
    return results

def get_geocode_info(row):
    """
    Constructs a full address from the row and uses OneMap API to retrieve
    postal code and coordinates.
    """
    # Construct the full address using block and street_name.
    # (You can modify this if you need to include town or other parts.)
    full_address = f"{row['block']} {row['street_name']}"
    # Replace spaces with '+' for URL encoding
    full_address_encoded = full_address.replace(" ", "+")
    
    results = collect_results(full_address_encoded)
    if results:
        # Choose the first result (assumed best match)
        res = results[0]
        # Extract postal code and coordinates (keys may be "POSTAL", "X", "Y")
        postal_code = res.get("POSTAL", "")
        longitude = res.get("LONGITUDE", "")
        latitude = res.get("LATITUDE", "")
        return pd.Series([postal_code, longitude, latitude])
    else:
        # No geocoding result found; return empty values.
        return pd.Series(["", "", ""])

In [ ]:
df = pd.read_csv('raw.csv')
# Apply the geocoding function to each row and save the results
df[['postal_code', 'longitude', 'latitude']] = df.progress_apply(get_geocode_info, axis=1)
df.to_csv('geocoded.csv', index=False)
print("Geocoding complete. Results saved to geocoded.csv")


100%|██████████| 199327/199327 [6:54:42<00:00,  8.01it/s]    


Geocoding complete. Results saved to geocoded.csv


# Generating Hawker Coordinates

In [48]:
def get_geocode_info_hawker(row):
    """
    Constructs a full address from the row and uses OneMap API to retrieve
    postal code and coordinates.
    """
    # delete everything after the last comma of the location_of_centre. there could be multiple commas in the address
    location_of_centre = row['location_of_centre']
    location_of_centre = location_of_centre.rsplit(',', 1)[0]

    # remove the leading letters 'Blk' if they exist
    location_of_centre = location_of_centre.replace('Blk', '')
    location_of_centre = location_of_centre.strip()
    
    # Replace spaces with '+' for URL encoding
    full_address_encoded = location_of_centre.replace(" ", "+")
    # print(full_address_encoded)
    results = collect_results(full_address_encoded)
    if results:
        # Choose the first result (assumed best match)
        res = results[0]
        # Extract postal code and coordinates (keys may be "POSTAL", "X", "Y")
        postal_code = res.get("POSTAL", "")
        longitude = res.get("LONGITUDE", "")
        latitude = res.get("LATITUDE", "")
        return pd.Series([postal_code, longitude, latitude])
    else:
        # No geocoding result found; return empty values.
        return pd.Series(["", "", ""])


In [49]:
# # Apply the geocoding function to each row and save the results
# df[['postal_code', 'longitude', 'latitude']] = df.progress_apply(get_geocode_info, axis=1)
# df.to_csv('test_out.csv', index=False)
# print("Geocoding complete. Results saved to geocoded.csv")

hawker_df = pd.read_csv('ListofGovernmentMarketsHawkerCentres.csv')
hawker_df[['postal_code', 'longitude', 'latitude']] = hawker_df.progress_apply(get_geocode_info_hawker, axis=1)
hawker_df.to_csv('hawker_geocoded.csv', index=False)
print("Geocoding complete. Results saved to hawker_geocoded.csv")

100%|██████████| 107/107 [11:52<00:00,  6.66s/it]


Geocoding complete. Results saved to hawker_geocoded.csv


# Generating School Coordinates

In [50]:
def get_geocode_info_hawker(row):
    """
    Constructs a full address from the row and uses OneMap API to retrieve
    postal code and coordinates.
    """
    address = row['address']
    
    # Replace spaces with '+' for URL encoding
    full_address_encoded = address.replace(" ", "+")
    # print(full_address_encoded)
    results = collect_results(full_address_encoded)
    if results:
        # Choose the first result (assumed best match)
        res = results[0]
        # Extract postal code and coordinates (keys may be "POSTAL", "X", "Y")
        postal_code = res.get("POSTAL", "")
        longitude = res.get("LONGITUDE", "")
        latitude = res.get("LATITUDE", "")
        return pd.Series([postal_code, longitude, latitude])
    else:
        # No geocoding result found; return empty values.
        return pd.Series(["", "", ""])


In [51]:
school_df = pd.read_csv('Generalinformationofschools.csv')
school_df[['postal_code', 'longitude', 'latitude']] = school_df.progress_apply(get_geocode_info_hawker, axis=1)
school_df.to_csv('school_geocoded.csv', index=False)
print("Geocoding complete. Results saved to school_geocoded.csv")

100%|██████████| 337/337 [22:16<00:00,  3.97s/it]  

Geocoding complete. Results saved to school_geocoded.csv


# Convert Park json data to csv data 
kiv for later, too much work right now

In [ ]:
# Convert Parks.geojson data to csv data
import json
import csv

with open('Parks.geojson') as f:
    data = json.load(f)

features = data['features']

# open a file for writing
park_data = open('park_data.csv', 'w')

# create the csv writer object
csvwriter = csv.writer(park_data)

count = 0
for feature in features:
    if count == 0:
        header = feature['properties'].keys()
        csvwriter.writerow(header)
        count += 1
    csvwriter.writerow(feature['properties'].values())

park_data.close()
print("Conversion complete. Results saved to park_data.csv")

Conversion complete. Results saved to park_data.csv


In [58]:
# Convert Parks X Y data to long lat data
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df = pd.read_csv('park_data.csv')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.X, df.Y))
gdf = gdf.to_crs("EPSG:4326")
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y
gdf.to_csv('park_data_long_lat.csv', index=False)
print("Conversion complete. Results saved to park_geocoded.csv")

ValueError: Cannot transform naive geometries.  Please set a crs on the object first.

# Combine datasets into a Facilities dataset

In [66]:
import pandas as pd

# Load each CSV file
hawker_df = pd.read_csv("hawker_geocoded.csv")
malls_df = pd.read_csv("malls_geocoded.csv")
mrt_df = pd.read_csv("mrt_geocoded.csv")
schools_df = pd.read_csv("school_geocoded.csv")
parks_df = pd.read_csv("parks_geocoded.csv")

# Extract relevant columns and add facility_type
hawker_df = hawker_df[['name_of_centre', 'longitude', 'latitude']].copy()
hawker_df.rename(columns={'name_of_centre': 'facility_name'}, inplace=True)
hawker_df['facility_type'] = 'hawker'

malls_df = malls_df[['Mall Name', 'LONGITUDE', 'LATITUDE']].copy()
malls_df.rename(columns={'Mall Name': 'facility_name', 'LONGITUDE': 'longitude', 'LATITUDE': 'latitude'}, inplace=True)
malls_df['facility_type'] = 'mall'

mrt_df = mrt_df[['STN_NAME', 'Longitude', 'Latitude']].copy()
mrt_df.rename(columns={'STN_NAME': 'facility_name', 'Longitude': 'longitude', 'Latitude': 'latitude'}, inplace=True)
mrt_df['facility_type'] = 'mrt'

schools_df = schools_df[['school_name', 'longitude', 'latitude']].copy()
schools_df.rename(columns={'school_name': 'facility_name'}, inplace=True)
schools_df['facility_type'] = 'school'

parks_df = parks_df[['NAME', 'longitude', 'latitude']].copy()
parks_df.rename(columns={'NAME': 'facility_name'}, inplace=True)
parks_df['facility_type'] = 'park'

# Combine all dataframes
facilities_df = pd.concat([hawker_df, malls_df, mrt_df, schools_df, parks_df], ignore_index=True)

# Reorder columns
facilities_df = facilities_df[['facility_type', 'facility_name', 'longitude', 'latitude']]

# Save to CSV
facilities_df.to_csv("facilities.csv", index=False)